# Event log 
Notebook for reading data from eventLog

In [1]:
import numpy as np
import pandas as pd
import h5py
import ast

eventArg  = {}
eventName = {}
eventlog  = {}
event = {}

path = '../datasets/'
key = '/events/'

fname = [
    'VRAcuityExp_2017-07-13_14-39-17_VR-4A_NIC',
    'VRAcuityExp_2017-07-13_15-05-16_VR-2B_NIC',
    'VRAcuityExp_2017-07-13_15-19-09_VR-2A_EDU',
    'VRAcuityExp_2017-07-13_15-38-34_VR-1A_NIC',
    'VRAcuityExp_2017-07-13_15-53-40_VR-1B_NIC',
    'VRAcuityExp_2017-07-13_16-11-46_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_16-27-08_VR-3A_NIC',
    'VRAcuityExp_2017-07-13_17-09-07_VR-5A_NIC', ]


for i, x in enumerate(fname):
    eventlog[i] = pd.read_hdf(path+x+'.h5', key+'eventlog')      
    #eventArg[i] = h5py.File(path+x+'.h5', 'r')
    #eventName[i] = pd.read_hdf(path+x+h5, key+'eventNames')


In [26]:
#eventArg analysis
hdf5_file_name = path+fname[0]+'.h5'
dataset_name   = '/events/'

file    = h5py.File(hdf5_file_name, 'r')   # 'r' means that hdf5 file is open in read-only mode
dataset = file[dataset_name]

eventN = dataset['eventNames']
eventA = dataset['eventArguments']

for i, x in enumerate(eventA):
    #print(type(x))
    #print(x, '\n')
    event[i] = x
    

In [32]:
for i, x in enumerate(eventA):
    event[i] = x

i=1
event[i] = event[i].decode('UTF-8')
print(event[i])
b = ast.literal_eval(event[i])
print(b)
print(b['duration'], '\n')
    
i=0
event[i] = event[i].decode('UTF-8')
x = event[i].replace('\'', '"')
#b = ast.literal_eval(x)
#print(b)



{'duration': 60.0}
{'duration': 60.0}
60.0 



In [110]:
for i, x in enumerate(eventA):
    event[i] = x

i = 3

for i in range(0, len(eventA)-1):
    event[i] = event[i].decode('UTF-8')
    x = event[i].replace('\'', '\"')
    #print(x, '\n')

    n = x.find('"attr"')
    if n != -1:
        x = x='{' + x[n:]
    
    print(i, x)
    b = ast.literal_eval(x)
    

0 {"attr": "visible", "value": False}
1 {"duration": 60.0}
2 {"attr": "visible", "value": True}
3 {"attr": "speed", "value": 7.0}
4 {"duration": 30.0}
5 {"attr": "visible", "value": False}
6 {"duration": 10.0}
7 {"attr": "visible", "value": True}
8 {"attr": "speed", "value": -7.0}
9 {"duration": 30.0}
10 {"attr": "visible", "value": False}
11 {"duration": 10.0}
12 {"attr": "visible", "value": True}
13 {"attr": "speed", "value": 0.0}
14 {"duration": 30.0}
15 {"attr": "visible", "value": False}
16 {"duration": 10.0}
17 {"attr": "visible", "value": True}
18 {"attr": "speed", "value": -0.0}
19 {"duration": 30.0}
20 {"attr": "visible", "value": False}
21 {"duration": 10.0}
22 {"attr": "visible", "value": True}
23 {"attr": "speed", "value": 7.0}
24 {"duration": 30.0}
25 {"attr": "visible", "value": False}
26 {"duration": 10.0}
27 {"attr": "visible", "value": True}
28 {"attr": "speed", "value": -7.0}
29 {"duration": 30.0}
30 {"attr": "visible", "value": False}
31 {"duration": 10.0}
32 {"attr"

ValueError: malformed node or string: <_ast.Call object at 0x7fcc02cb2128>

In [25]:
import json
json_string1 = '{"favorited": false, "contributors": null}'
json_string = "{'favorited': false, 'contributors': null}"
s = json_string.replace('\'', '"')
print(s)
value = json.loads(s)
print(type(value))

value1 = json.loads(json_string1)
print(type(value1))


{'favorited': false, 'contributors': null}
{"favorited": false, "contributors": null}
<class 'dict'>
<class 'dict'>
